# Imports


In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import dateutil.parser
from statistics import mean
import math
import csv
import tkinter
from tkinter import filedialog
import os
from tkinter import * 
from tkinter.ttk import *
from tkinter.filedialog import askopenfile
import os

# GUI

In [2]:
# code copied from : Huseyn Mammadli , https://stackoverflow.com/questions/19944712/browse-for-file-path-in-python/66699008#66699008

root = Tk()
root.title("KPIs Comparators")
root.geometry('700x600')
path = ""

def open_file(): 
    file = askopenfile(mode ='r', filetypes =[('csv Files', '*.csv')])
    if file is not None:
        global path
        path = file.name
        root.destroy()
    
     

btn = Button(root, text ='Browse File Directory', command =lambda:open_file())
btn.pack(side = TOP, pady = 10) 

mainloop() 
folder_path = os.path.dirname(path)
final_csv_path = folder_path + "/Benchmarked_KPIs.csv"

# Cell Statistics

In [3]:
raw_KPIs= np.genfromtxt(path, delimiter = ',', skip_header = 1, autostrip = True, dtype = np.str)
headers = np.genfromtxt(path, delimiter = ',', autostrip = True, dtype = np.str)[0].tolist()
# sorting KPIs alphabetically to the granularity
sorted_KPIs = raw_KPIs[np.lexsort((raw_KPIs[:,1],raw_KPIs[:,0]))]
sorted_KPIs


C:\Users\yassi\AppData\Local\Temp/ipykernel_14600/593265475.py:1: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  raw_KPIs= np.genfromtxt(path, delimiter = ',', skip_header = 1, autostrip = True, dtype = np.str)
C:\Users\yassi\AppData\Local\Temp/ipykernel_14600/593265475.py:2: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  headers = np.genfromtxt(path, delimi

array([['apple', '10/19/2022', '10/25/2022', '4.1', '99'],
       ['apple', '10/20/2022', '10/25/2022', '4.1', '67'],
       ['apple', '10/21/2022', '10/25/2022', '4.1', '23'],
       ['apple', '10/22/2022', '10/25/2022', '', ''],
       ['apple', '10/23/2022', '10/25/2022', '5.1', '89'],
       ['apple', '10/24/2022', '10/25/2022', '4.1', '98'],
       ['apple', '10/25/2022', '10/25/2022', '3.1', '97'],
       ['apple', '10/26/2022', '10/25/2022', '3.1', '44'],
       ['apple', '10/27/2022', '10/25/2022', '2.1', '33'],
       ['apple', '10/28/2022', '10/25/2022', '2.1', '89'],
       ['apple', '10/29/2022', '10/25/2022', '1.1', '92'],
       ['apple', '10/30/2022', '10/25/2022', '4.1', '91'],
       ['orange', '10/19/2022', '10/23/2022', '', '100'],
       ['orange', '10/20/2022', '10/23/2022', '3', '22'],
       ['orange', '10/21/2022', '10/23/2022', '4', '22'],
       ['orange', '10/22/2022', '10/23/2022', '2', '22'],
       ['orange', '10/23/2022', '10/23/2022', '2', '22'],
       

# Date preprocessing

In [4]:
def get_delta_days(day1,day2):
    """returns the difference between two dates in (days) unit """
    day1 = get_formatted_date(day1)
    day2 = get_formatted_date(day2)

    
    delta = abs(day2 - day1)
    return int(delta.days)

def get_formatted_date(date_str):
    """parse the string date and returns the date in the python conventional date format"""
    date = dateutil.parser.parse(date_str)
    return datetime(date.year, date.month, date.day)
    
x = '10/19/2022'
y = "2022-10-23"




# Importing  KPIs

In [5]:
# storing cell_name, number_of_occurences, index_of_first_Occurence,index_of_activity_date --> "cells_occurences"
cells_occurences = [[]]
""" for cell in sorted_KPIs[:,0]:
    count = np.count_nonzero(sorted_KPIs[:,0] == cell)
    cells_occurences.append([cell, count, list(sorted_KPIs[:,0]).index(cell)])"""

for i,cell in enumerate(sorted_KPIs[:,0:1]):
    count = np.count_nonzero(sorted_KPIs[:,0] == cell[0])
    index_of_first_Occurence = list(sorted_KPIs[:,0]).index(cell[0])
    #index_of_event = list(sorted_KPIs[:,1].index())
    cells_occurences.append([cell[0], count,index_of_first_Occurence ])  
    
# tupling the matrix to check for uniqueness
new_array = [tuple(row) for row in cells_occurences]
cells_occurences = np.unique(new_array)

#De-tupling and removing the first empty entry
cells_occurences = [list(item) for item in cells_occurences]
cells_occurences = cells_occurences[1:][:]
cells_occurences

#add the number of days from the first provided KPI date for each cell until the activity date
for i,cell in enumerate(cells_occurences):
    
    index = cell[2]
    date1 = sorted_KPIs[index][1]
    date2 = sorted_KPIs[index][2]
    activity_span = (get_delta_days(date1,date2))
    cells_occurences[i].append(activity_span)

cells_occurences

P:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


[['apple', 12, 0, 6], ['orange', 12, 12, 4]]

# Converting string data to numeric 

Slicing the whole sorted KPIs by removing the Date and Cells after sorting them according to the date then cell.
However, the Numpy and pandas can not cast string matrix to float, therefore, an iterative approach was used with "pd.to_numeric" and then retransforming the pd frame to the numpy array while preserving the order and avoiding the preprocessing again

https://stackoverflow.com/questions/16223483/forced-conversion-of-non-numeric-numpy-arrays-with-nan-replacement
https://stackoverflow.com/questions/68907033/how-to-convert-a-2d-array-of-strings-and-numbers-to-a-numpy-float-array

In [6]:
#numeric_KPIs = sorted_KPIs[1:,2:]
numeric_KPIs = sorted_KPIs[:,3:]
numeric_KPIs =  [pd.to_numeric(col, errors='coerce') for col in numeric_KPIs ] 
numeric_KPIs = np.asarray(numeric_KPIs)
numeric_KPIs

array([[  4.1,  99. ],
       [  4.1,  67. ],
       [  4.1,  23. ],
       [  nan,   nan],
       [  5.1,  89. ],
       [  4.1,  98. ],
       [  3.1,  97. ],
       [  3.1,  44. ],
       [  2.1,  33. ],
       [  2.1,  89. ],
       [  1.1,  92. ],
       [  4.1,  91. ],
       [  nan, 100. ],
       [  3. ,  22. ],
       [  4. ,  22. ],
       [  2. ,  22. ],
       [  2. ,  22. ],
       [  2. ,  22. ],
       [  4. ,  22. ],
       [  5. ,  22. ],
       [  6. ,  22. ],
       [  7. ,  22. ],
       [  2. ,  45. ],
       [  6.1,  56. ]])

# Voting for Success Rate or Drop Rate

In [7]:
def recognize_data_optimality(data):
    """ sums up the value in each column and if sum is above 40 then it is a success rate data, otherwise it is a drop rate"""
    voters=[]
    no_cols = len(data[0])
    no_rows = len(data)
    for j in range(no_cols):
        negative_counter = 0
        positive_counter = 0
        
        for i in range(no_rows):
            if data[i][j] == "nan":
                continue
                
            if data[i][j] > 30:
                positive_counter +=1
            else:
                negative_counter +=1
        # voter = 1 --> success rate
        # voter = 0 --> drop rate
        voter = 1 if positive_counter > negative_counter else 0
        voters.append(voter)
    return voters


#  voter = 1 --> success rate / voter = 0 --> drop rate
voters = recognize_data_optimality(numeric_KPIs)
voters

[0, 1]

# Replacing the NANs 

In [8]:
# Replace the NAN values with 0 if it's drop rate and with 1 if it's success rate
def replace_nan(data):
    no_cols = len(data[0])
    no_rows = len(data)
    for j in range(no_cols):
        if voters[j] == 1:
            np.nan_to_num(data[:,j], copy=False, nan=100, posinf=100, neginf=0)
        else:
            np.nan_to_num(data[:,j], copy=False, nan=0, posinf=100, neginf=0)
    
replace_nan(numeric_KPIs)
numeric_KPIs

array([[  4.1,  99. ],
       [  4.1,  67. ],
       [  4.1,  23. ],
       [  0. , 100. ],
       [  5.1,  89. ],
       [  4.1,  98. ],
       [  3.1,  97. ],
       [  3.1,  44. ],
       [  2.1,  33. ],
       [  2.1,  89. ],
       [  1.1,  92. ],
       [  4.1,  91. ],
       [  0. , 100. ],
       [  3. ,  22. ],
       [  4. ,  22. ],
       [  2. ,  22. ],
       [  2. ,  22. ],
       [  2. ,  22. ],
       [  4. ,  22. ],
       [  5. ,  22. ],
       [  6. ,  22. ],
       [  7. ,  22. ],
       [  2. ,  45. ],
       [  6.1,  56. ]])

# Outlier check

Return array where sudden peaks in data sequence is removed, to avoid deviating the mean significantly while considering which type of peaks should be removed (+ve) or (-ve) peaks by sorting the array ascendingly or descendingly and removing element by element while comparing the new avg with original avg and removing the element that caused big difference between the old and new avg. The function accounts of low samples number, where if the reduced array size is half of the original array,then the data is fluctuating heavily, and function will return original data 
https://math.stackexchange.com/questions/22348/how-to-add-and-subtract-values-from-an-average

In [9]:
def outlier_check(old,KPI_type = 0,tolerance_rate =10, new_size_limit = 2 ):

    old_avg = mean(old)
    l = len(old)
    if l < 5:
        return old
    
    tolerance = tolerance_rate/100 * old_avg
    
    # new array returned after removing the outliers
    modified = old.copy()
    # counter of how many elements were removed from the orignial array
    rmv_cnt =0
    

    
    if KPI_type == 1:
        # KPI_type = 1, means the KPI is success rate, and the function will remove the low values
        new = sorted(old, reverse=False)
        condition = lambda delta:delta < tolerance
    else:
        # KPI_type = 0, means the KPI is drop rate, and the function will remove the high values
        new = sorted(old, reverse=True)
        condition = lambda delta:delta > tolerance
    
    for num in new:
        new_avg = old_avg - (num - old_avg)/l
        if condition(abs(new_avg - old_avg)):
            modified.remove(num)
            rmv_cnt +=1
            if rmv_cnt < int(l/new_size_limit):
                # return the original array iff the removed elements count are more than half of the whole array elements count
                return old
        else:
            break
 
    
    
    return modified
d = [2,7,3,2,2,2,3,7]
d2 = [2,3,2,3,2,3]
d3 = [1,2,3]
n = outlier_check(d3,KPI_type = 0,tolerance_rate =10, new_size_limit = 10)
mean(n)
print("old:",d3)
print("new:",n)

old: [1, 2, 3]
new: [1, 2, 3]


# Improvement Rate

In [10]:
def benchmark(pre,post, KPI_type = 1):
    pre_mean = mean(pre)
    post_mean = mean(post)
    if KPI_type == 1:
        # Success rate KPIs
        delta = post_mean - pre_mean
    else:
        # Drop rate KPIs
        delta = -1 * (post_mean - pre_mean)
        
    return  100 * (delta / pre_mean)
benchmark([1,2,3],[5,6,7],1)

200.0

# Comparison

In [11]:
cells_benchmarked_KPIs=[]
for i,cell in enumerate(cells_occurences):
    
    cell_name,length,start,mid = cell
    benchmarked_KPIs = []
    
    for j in range(len(numeric_KPIs[0])):
        
        raw_pre_KPIs = numeric_KPIs[start:start+mid+1,j].tolist()
        raw_post_KPIs = numeric_KPIs[start+mid+1:start+length,j].tolist()
        
        pre_KPIs = outlier_check(raw_pre_KPIs,KPI_type = voters[j],tolerance_rate =10, new_size_limit = 2)
        post_KPIs = outlier_check(raw_post_KPIs,KPI_type = voters[j],tolerance_rate =10, new_size_limit = 2)
        
        benchmarked_KPI = benchmark(pre =pre_KPIs, post = post_KPIs, KPI_type = voters[j])
        benchmarked_KPIs.append(round(benchmarked_KPI,1))
        
        print(f"{cell_name} KPI of {voters[j]} is : {benchmarked_KPI}")
        
    cells_benchmarked_KPIs.append(benchmarked_KPIs)
        



apple KPI of 0 is : 28.86178861788618
apple KPI of 1 is : -14.729493891797565
orange KPI of 0 is : -108.4415584415584
orange KPI of 1 is : -19.832826747720368


# Preparing the final benchmark Matrix

In [12]:
all_benchmarks = [[headers[0]] + headers[3:]]
for i in range(len(cells_occurences)):
    # cell name + the benchmarked KPIs
    data = [cells_occurences[i][0]] +cells_benchmarked_KPIs[i]
    all_benchmarks.append(data)

all_benchmarks[0].append("Status")
# Status check for each cell
for cell in all_benchmarks[1:]:
    if all(KPI > 0 for KPI in cell[1:]):
        status = "Improved"
    elif all(KPI == 0 for KPI in cell[1:]):
        status = "Stable"
    else:
        status = "Degraded"
    cell.append(status)
    


# Saving the CSV file

In [13]:
KPIs_df = pd.DataFrame(all_benchmarks)
KPIs_df.to_csv(final_csv_path, index=False, header=False)

In [14]:
all_benchmarks

[['Cell', 'price', 'SR', 'Status'],
 ['apple', 28.9, -14.7, 'Degraded'],
 ['orange', -108.4, -19.8, 'Degraded']]